# Project — Part I (Bootstrapping Swap Curves) #

In the IR Data.xlsm spreadsheeet, OIS data is provided. Bootstrap the
OIS discount factor Do(0, T ) and plot the discount curve for T ∈ [0, 30].

* Day Count Convention	30/360
* O/N Leg Frequency	Daily
* Fixed Leg Frequency	Annual


In [264]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# handy it seems
# https://docs.sympy.org/latest/modules/solvers/solvers.html
from sympy.solvers import solve
from sympy import Symbol


# read data
ois_data = pd.read_csv("../data/OIS_Data.csv")
irs_data = pd.read_csv("../data/IRS_Data.csv")
ois_data.columns = map(str.lower, ois_data.columns)
irs_data.columns = map(str.lower, irs_data.columns)

# use dict comprehension?
tenor_mapping = {
    "6m": 0.5,
    "1y": 1,
    "2y": 2,
    "3y": 3,
    "4y": 4,
    "5y": 5,
    "7y": 7,
    "10y": 10,
    "15y": 15,
    "20y": 20,
    "30y": 30,
}

# OIS processing
ois_data["tenor"] = ois_data["tenor"].map(tenor_mapping)
ois_data["rate"] = ois_data["rate"].str.strip("%").astype(float) / 100.0

# DAY COUNT CONVENTION IS 30/360
FULL_YEAR = 360

In [265]:
ois_data["tenor_diff"] = ois_data["tenor"] - ois_data["tenor"].shift()

##  solve for < 1 y stuff ##

Because these 2 are anchors i believe

In [266]:
f_6m = Symbol("f_6m")
f_6m = solve((1 + 0.5 * ois_data.loc[0, "rate"]) ** (1 / 180) - (1 + f_6m / 360))[0]
print(f_6m)

0.00249844747059136


In [267]:
f_1y = Symbol("f_1y")
f_1y = solve(
    ((1 + 1 * ois_data.loc[1, "rate"]) / (1 + f_6m / 360) ** 180) ** (1 / 180)
    - (1 + f_1y / 360)
)[0]
print(f_1y)

0.00349259610064934


In [268]:
ois_data.loc[0, "disc_factor"] = 1 / (1 + f_6m / 360) ** 180
ois_data.loc[1, "disc_factor"] = 1 / (
    ((1 + f_6m / 360) ** 180) * ((1 + f_1y / 360) ** 180)
)

In [269]:
ois_data

,tenor,product,rate,tenor_diff,disc_factor
0,0.5,OIS,0.00250,NaN,0.998751560549318
1,1.0,OIS,0.00300,0.5,0.997008973080741
2,2.0,OIS,0.00325,1.0,NaN
3,3.0,OIS,0.00335,1.0,NaN
4,4.0,OIS,0.00350,1.0,NaN
5,5.0,OIS,0.00360,1.0,NaN
6,7.0,OIS,0.00400,2.0,NaN
7,10.0,OIS,0.00450,3.0,NaN
8,15.0,OIS,0.00500,5.0,NaN
9,20.0,OIS,0.00525,5.0,NaN


## Let's try the collapsing shit, thx eko ##
I can test up until 5 first

In [270]:
y_1_funny = (
    ((1 + f_6m / 360) ** 180)
    * ((1 + f_1y / 360) ** 180)
    * ois_data.loc[1, "disc_factor"]
)
# youd be surprised
print(y_1_funny)

1.00000000000000


In [271]:
for k in range(2, 6):
    denominator = 1 / (1 + 1 * ois_data.loc[k, "rate"])
    sum_prev_disc_rate = ois_data.loc[1:k, "disc_factor"].sum()
    # print(sum_prev_disc_rate)
    numerator = y_1_funny - 1 * ois_data.loc[k, "rate"] * sum_prev_disc_rate
    ois_data.loc[k, "disc_factor"] = numerator * denominator

## interpolation required ##

In [272]:
previous_disc_factors = ois_data.loc[1:5, "disc_factor"].to_numpy()
print(previous_disc_factors)

[0.997008973080741 0.993530745913270 0.990015141218289 0.986116649715253
 0.982184119733221]


In [273]:
# k = 6 to 10; try alternative method
for k in range(6, 11):
    mul_fact = 1.0 / ois_data.loc[k, "tenor_diff"]
    rate = ois_data.loc[k, "rate"]
    max_sum = sum(range(1, int(ois_data.loc[k, "tenor_diff"])))
    # print(f"{mul_fact} * {max_sum} = {mul_fact * max_sum}")
    regressor = mul_fact * max_sum
    pv_fix_wo_unknown = (
        previous_disc_factors.sum() + regressor * previous_disc_factors[-1]
    )
    pv_float_wo_unknown = y_1_funny
    x = Symbol("x")
    tosolve = solve(
        rate * 1 * (pv_fix_wo_unknown + regressor * x + x) - (pv_float_wo_unknown - x)
    )
    # print(tosolve[0])
    ois_data.loc[k, "disc_factor"] = tosolve[0]

    # find points between
    interpolated_disc_factors = []
    diff_factor = mul_fact * (
        ois_data.loc[k, "disc_factor"] - ois_data.loc[k - 1, "disc_factor"]
    )

    for x in range(1, int(ois_data.loc[k, "tenor_diff"])):
        to_append = ois_data.loc[k - 1, "disc_factor"] + diff_factor * x
        # print(f"{x} = {to_append}")
        interpolated_disc_factors.append(to_append)
    # append them
    previous_disc_factors = np.append(previous_disc_factors, interpolated_disc_factors)
    previous_disc_factors = np.append(
        previous_disc_factors, ois_data.loc[k, "disc_factor"]
    )

In [274]:
ois_disc_factors = np.append(ois_data["disc_factor"].values[0], previous_disc_factors)
ois_tenors = np.append([0.5], np.arange(1, 31, 1))
df_ois_df = pd.DataFrame({"tenor": ois_tenors, "ois_df": ois_disc_factors})

In [275]:
df_ois_df

,tenor,ois_df
0,0.5,0.998751560549318
1,1.0,0.997008973080741
2,2.0,0.993530745913270
3,3.0,0.990015141218289
4,4.0,0.986116649715253
5,5.0,0.982184119733221
6,6.0,0.977294947163773
7,7.0,0.972405774594324
8,8.0,0.966929475904951
9,9.0,0.961453177215578


### finally OIS data

In [276]:
ois_data

,tenor,product,rate,tenor_diff,disc_factor
0,0.5,OIS,0.00250,NaN,0.998751560549318
1,1.0,OIS,0.00300,0.5,0.997008973080741
2,2.0,OIS,0.00325,1.0,0.993530745913270
3,3.0,OIS,0.00335,1.0,0.990015141218289
4,4.0,OIS,0.00350,1.0,0.986116649715253
5,5.0,OIS,0.00360,1.0,0.982184119733221
6,7.0,OIS,0.00400,2.0,0.972405774594324
7,10.0,OIS,0.00450,3.0,0.955976878526205
8,15.0,OIS,0.00500,5.0,0.927611479605330
9,20.0,OIS,0.00525,5.0,0.900075937041340


### 1b

In [277]:
# combined data
combined_tenors = np.arange(0.5, 30.5, 0.5)
df_combined = pd.DataFrame({"tenor": combined_tenors})
df_combined = pd.merge(df_combined, df_ois_df, how="left", on="tenor")
df_combined["ois_df"] = df_combined["ois_df"].astype(float)
df_combined.interpolate(inplace=True)
# df_combined = pd.merge(df_combined, ois_data[["tenor", "rate"]], how="left", on="tenor")
# df_combined.rename(columns={"rate": "ois_rate"}, inplace=True)

In [278]:
# read data
irs_data = pd.read_csv("../data/IRS_Data.csv")
irs_data.columns = map(str.lower, irs_data.columns)
irs_data["tenor"] = irs_data["tenor"].map(tenor_mapping)
irs_data["rate"] = irs_data["rate"].str.strip("%").astype(float) / 100.0
irs_data["tenor_diff"] = irs_data["tenor"] - irs_data["tenor"].shift()
irs_data["tenor_diff"].fillna(0, inplace=True)
irs_data["interp_count"] = (irs_data["tenor_diff"] / 0.5).astype(int)
irs_data.rename(columns={"rate": "irs_rate"}, inplace=True)

In [279]:
irs_data.loc[0, "irs_df"] = 1 / (
    1 + irs_data.loc[0, "tenor"] * irs_data.loc[0, "irs_rate"]
)
irs_data.loc[0, "fw_libor"] = irs_data.loc[0, "irs_rate"]

In [280]:
irs_data

,tenor,product,irs_rate,tenor_diff,interp_count,irs_df,fw_libor
0,0.5,LIBOR,0.0250,0.0,0,0.987654,0.025
1,1.0,IRS,0.0280,0.5,1,NaN,NaN
2,2.0,IRS,0.0300,1.0,2,NaN,NaN
3,3.0,IRS,0.0315,1.0,2,NaN,NaN
4,4.0,IRS,0.0325,1.0,2,NaN,NaN
5,5.0,IRS,0.0330,1.0,2,NaN,NaN
6,7.0,IRS,0.0350,2.0,4,NaN,NaN
7,10.0,IRS,0.0370,3.0,6,NaN,NaN
8,15.0,IRS,0.0400,5.0,10,NaN,NaN
9,20.0,IRS,0.0450,5.0,10,NaN,NaN


In [281]:
# finally something we can work with
df_combined = pd.merge(
    df_combined,
    irs_data[["tenor", "irs_rate", "fw_libor", "irs_df"]],
    how="left",
    on="tenor",
)

In [282]:
irs_df_1 = Symbol("irs_df_1")
just_try_irs_df_1 = solve(
    (
        0.5
        * (df_combined.loc[0, "ois_df"] + df_combined.loc[1, "ois_df"])
        * df_combined.loc[1, "irs_rate"]
    )
    - 0.5
    * (
        df_combined.loc[0, "ois_df"] * df_combined.loc[0, "fw_libor"]
        + df_combined.loc[1, "ois_df"]
        * 2
        * (((df_combined.loc[0, "irs_df"] - irs_df_1) / irs_df_1))
    )
)

In [283]:
# define a root search function(bisect)
def bisect_func(f, a, b, tol):
    assert a < b and f(a) * f(b) < 0
    c = (a + b) / 2
    while (b - a) / 2 > tol:
        c = (a + b) / 2
        if abs(f(c)) < tol:
            return c
        else:
            if f(a) * f(c) < 0:
                b = c
            else:
                a = c
    return c

In [284]:
def get_pv_float(x):
    start_irs_df = df_combined.loc[prev_idx_with_k, "irs_df"]
    df_list = [start_irs_df]
    # print(f"INSIDE GET FLOAT > {prev_idx_with_k} & {idx_with_k} and {pv_float_known}")
    max_count = idx_with_k - prev_idx_with_k
    if abs(max_count) > 1:
        diff = x - start_irs_df
        libor_payoff_list = []
        for n in range(1, max_count + 1):
            idx_to_use = prev_idx_with_k + n
            # print(
            #    f"INSIDE GET FLOAT > {prev_idx_with_k}- {df_list} & {idx_with_k} and {pv_float_known} and {idx_to_use}"
            # )

            mul_fact = n / (max_count)
            df_interp = start_irs_df + mul_fact * diff
            libor_payoff = (
                0.5
                * (2)
                * ((df_list[-1] - df_interp) / df_interp)
                * df_combined.loc[idx_to_use, "ois_df"]
            )
            df_list.append(df_interp)
            libor_payoff_list.append(libor_payoff)
        pv_float_unknown = sum(libor_payoff_list)

    else:
        df_interp = x
        libor_payoff = (
            0.5
            * (2)
            * ((df_list[-1] - df_interp) / df_interp)
            * df_combined.loc[1, "ois_df"]
        )
        pv_float_unknown = libor_payoff

    return pv_float_unknown + pv_float_known - pv_fix

In [285]:
# start from 1 because we already know for row 0, i.e. 6m
idx_with_k = 1
prev_idx_with_k = 0
maxrow = len(df_combined)
while idx_with_k < maxrow:
    if np.isnan(df_combined.loc[idx_with_k, "irs_rate"]):
        # print("next")
        idx_with_k = idx_with_k + 1
    else:
        # present value of fix
        pv_fix = (
            0.5
            * df_combined.loc[idx_with_k, "irs_rate"]
            * df_combined.loc[0:idx_with_k, "ois_df"].sum()
        )
        # print(df_combined.loc[0:idx_with_k, "ois_df"])

        # we try to sum up floating leg as much as possible
        pv_float_known = 0
        idx_with_fwlibor = 0
        k = 0
        while not (np.isnan(df_combined.loc[k, "fw_libor"])):
            known_float = (
                0.5 * df_combined.loc[k, "ois_df"] * df_combined.loc[k, "fw_libor"]
            )
            pv_float_known = pv_float_known + known_float
            idx_with_fwlibor = idx_with_fwlibor + 1
            k = k + 1
        print(f"known until {k-1}")
        # refer senior for inspiration how to do a very challenging interpolation
        print(f"{prev_idx_with_k} & {idx_with_k} and and {pv_fix} vs {pv_float_known}")
        # f = lambda x: (get_pv_float(x))
        f = lambda x: (get_pv_float(x))
        df_last = bisect_func(f, 1e-10, 1, 1e-10)
        print(df_last)

        # fill in value !
        max_count = idx_with_k - prev_idx_with_k
        if (max_count) > 1:
            for n in range(prev_idx_with_k + 1, idx_with_k + 1):
                start_irs_df = df_combined.loc[prev_idx_with_k, "irs_df"]
                diff = df_last - start_irs_df
                print(f"start from {start_irs_df} with diff {diff}")
                mul_fact = (n - prev_idx_with_k) / (max_count)
                df_interp = start_irs_df + mul_fact * diff
                print(n)
                df_combined.loc[n, "irs_df"] = df_interp
                df_combined.loc[n, "fw_libor"] = 2 * (
                    df_combined.loc[n - 1, "irs_df"] / df_combined.loc[n, "irs_df"] - 1
                )
                print("filled !")

        else:
            df_combined.loc[1, "irs_df"] = df_last
            df_combined.loc[1, "fw_libor"] = 2 * (
                df_combined.loc[0, "irs_df"] / df_combined.loc[1, "irs_df"] - 1
            )

        prev_idx_with_k = idx_with_k
        idx_with_k = idx_with_k + 1

known until 0
0 & 1 and and 0.02794064747082083 vs 0.012484394506866479
0.9725768303060389
known until 1
1 & 3 and and 0.05976841708560501 vs 0.02794064745541936
0.9421790053660128
start from 0.9725768303060389 with diff -0.030397824940026075
2
filled !
start from 0.9725768303060389 with diff -0.030397824940026075
3
filled !
known until 3
3 & 5 and and 0.09397000027523435 vs 0.059768417142381985
0.9104815318804345
start from 0.9421790053660128 with diff -0.03169747348557839
4
filled !
start from 0.9421790053660128 with diff -0.03169747348557839
5
filled !
known until 5
5 & 7 and and 0.12903364124635444 vs 0.09397000030718423
0.8789806787511283
start from 0.9104815318804345 with diff -0.03150085312930617
6
filled !
start from 0.9104815318804345 with diff -0.03150085312930617
7
filled !
known until 7
7 & 9 and and 0.1634632935121541 vs 0.1290336413133756
0.8489892888033899
start from 0.8789806787511283 with diff -0.029991389947738423
8
filled !
start from 0.8789806787511283 with diff -0.

In [286]:
df_combined

,tenor,ois_df,irs_rate,fw_libor,irs_df
0,0.5,0.998752,0.0250,0.025000,0.987654
1,1.0,0.997009,0.0280,0.031005,0.972577
2,1.5,0.995270,NaN,0.031751,0.957378
3,2.0,0.993531,0.0300,0.032263,0.942179
4,2.5,0.991773,NaN,0.034218,0.926330
5,3.0,0.990015,0.0315,0.034814,0.910482
6,3.5,0.988066,NaN,0.035207,0.894731
7,4.0,0.986117,0.0325,0.035838,0.878981
8,4.5,0.984150,NaN,0.034713,0.863985
9,5.0,0.982184,0.0330,0.035326,0.848989


## GOD FINALLY

In [289]:
irs_data.drop(columns=["irs_df", "fw_libor"], inplace=True)
irs_data = pd.merge(irs_data, df_combined, how="left", on="tenor")
irs_data.drop(columns=["irs_rate_y"], inplace=True)

In [ ]:
ois_data

In [288]:
irs_data

,tenor,product,irs_rate_x,tenor_diff,interp_count,ois_df,irs_rate_y,fw_libor,irs_df
0,0.5,LIBOR,0.0250,0.0,0,0.998752,0.0250,0.025000,0.987654
1,1.0,IRS,0.0280,0.5,1,0.997009,0.0280,0.031005,0.972577
2,2.0,IRS,0.0300,1.0,2,0.993531,0.0300,0.032263,0.942179
3,3.0,IRS,0.0315,1.0,2,0.990015,0.0315,0.034814,0.910482
4,4.0,IRS,0.0325,1.0,2,0.986117,0.0325,0.035838,0.878981
5,5.0,IRS,0.0330,1.0,2,0.982184,0.0330,0.035326,0.848989
6,7.0,IRS,0.0350,2.0,4,0.972406,0.0350,0.041298,0.784216
7,10.0,IRS,0.0370,3.0,6,0.955977,0.0370,0.044033,0.692710
8,15.0,IRS,0.0400,5.0,10,0.927611,0.0400,0.051401,0.551081
9,20.0,IRS,0.0450,5.0,10,0.900076,0.0450,0.069994,0.408218
